In [1]:
import pandas as pd
import numpy as np
import json
import os
from matplotlib import pyplot as plt

import datasets

In [2]:
with open("extra_dataset/chat_gpt/dataset") as file:
    data = file.readlines()
data: list[str] = [
    i.strip() for i in data if i.strip()
]
data_dyu = [
	i.replace("Dyula:", "").strip() for i in data if "Dyula:" in i
]
data_fr = [
	i.replace("French:", "").strip() for i in data if "French:" in i
]

In [3]:
data_dyu[:5]

['Aw ni ce.', 'I ka kɛnɛ wa?', 'Mɔgɔ bɛ minɛ?', "N b'a fɛ.", 'A bɛ se ka taa?']

In [4]:
data_fr[:5]

['Merci beaucoup.',
 'Comment ça va?',
 'Où est la personne?',
 "Je l'aime.",
 'Peut-il partir?']

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer, M2M100ForConditionalGeneration, pipeline

checkpoint = "facebook/nllb-200-distilled-600M"


src_lang = "dyu_Latn"
tgt_lang = "fra_Latn"

tokenizer: NllbTokenizer = AutoTokenizer.from_pretrained(checkpoint, src_lang=src_lang, tgt_lang=tgt_lang)

model: M2M100ForConditionalGeneration = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pipe = pipeline('translation', model=model, tokenizer=tokenizer, src_lang=src_lang, tgt_lang=tgt_lang)

In [6]:
id(pipe.model)

140249654238752

In [7]:
from transformers import TranslationPipeline

In [8]:
def make_prediction(index = 0):
	query = data_dyu[index]
	
	tokenizer.src_lang = src_lang
	tokenizer.tgt_lang = tgt_lang

	model.generation_config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
	tokenizer
	inputs = tokenizer(query, return_tensors="pt").input_ids
	outputs = model.generate(inputs, max_new_tokens=300, do_sample=True, top_k=20, top_p=0.95) #  top_k=20, top_p=0.97

	print(query)
	print(tokenizer.batch_decode(inputs)[0])
	print(tokenizer.decode(outputs[0], skip_special_tokens=True))
	print(data_fr[index])

In [9]:
make_prediction(10)

I y'a kɛnɛ wa?
dyu_Latn I y'a kɛnɛ wa?</s>
Avez- vous guéri votre malade?
Comment allez-vous?


In [10]:
make_prediction(100)

N t'a boli kɛ, kunɔ n'a boli kɛ kun dɔ.
dyu_Latn N t'a boli kɛ, kunɔ n'a boli kɛ kun dɔ.</s>
C'est pour moi un bonheur, un gain de cause.
Je vais finir le travail aujourd'hui, mais je vais commencer un nouveau demain.


In [11]:
make_prediction(7)

I tɛ se ka fɔ.
dyu_Latn I tɛ se ka fɔ.</s>
En effet, il est impossible de s'y attendre.
Tu ne peux pas parler.


In [12]:
make_prediction(15)

I tɛ fɛere.
dyu_Latn I tɛ fɛere.</s>
Elle est très gentille.
Tu n'as pas raison.


In [13]:
make_prediction(25)

A bɛ se ka don.
dyu_Latn A bɛ se ka don.</s>
Il peut rentrer.
Il peut comprendre.


In [14]:
import random

for i in random.sample(range(len(data_dyu)), k=25):
    print("Index", i)
    make_prediction(i)
    print()

Index 22
A y'a taa.
dyu_Latn A y'a taa.</s>
Il s'en est allé.
Il est parti.

Index 241
N bɛ se ka taa sɛnɛ kɛ.
dyu_Latn N bɛ se ka taa sɛnɛ kɛ.</s>
Je peux aller travailler à la ferme.
Je vais partir demain.

Index 178
I ka taa ka bɔ ne, kunɔ i tɛ se ka taa.
dyu_Latn I ka taa ka bɔ ne, kunɔ i tɛ se ka taa.</s>
Eloigne-toi de moi! Levé, tu ne peux t'en aller!
Tu veux venir avec moi, mais tu ne peux pas partir.

Index 155
N y'a se ka kan kunɔ, n tɛ se ka fɔ.
dyu_Latn N y'a se ka kan kunɔ, n tɛ se ka fɔ.</s>
Je pourrais même me lever, mais je n'en peux rien dire.
Je veux comprendre, mais je ne veux pas parler.

Index 255
N ka taa kunɔ, kunɔ n tɛ se ka fɔ.
dyu_Latn N ka taa kunɔ, kunɔ n tɛ se ka fɔ.</s>
Je me réveille, et je ne sais pas dire où je me suis réveillé, et je me lève pour dire où j'ai été.
Je vais partir maintenant, mais je ne veux pas.

Index 230
N bɛ sɔrɔ taa kunɔ.
dyu_Latn N bɛ sɔrɔ taa kunɔ.</s>
Je me lève à la recherche de Dieu et de ce qu'il y a de mieux.
Je suis prêt à p